## Analyse der Publikationsjahre

### Vorbereitung

In [ ]:
# Importieren der benötigten Bibliotheken: 
import pandas as pd
from lxml import etree
import unicodedata
import plotly.express as px
import plotly.io as pio

In [ ]:
# Laden der MARC-xml-Datei in eine Liste "records": 
tree = etree.parse('dataset_tutorial_eco.xml')
root = tree.getroot()                
ns = {'marc': 'http://www.loc.gov/MARC21/slim'} 
records = root.findall('.//marc:record', namespaces=ns)
print("Gefundene Records:", len(records))

In [ ]:
# Funktion zum Extrahieren von Datensätzen
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    jahr = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    jahr008 = extract_text("marc:controlfield[@tag='008']")[7:11]   

    return {
        "idn": idn,
        "author": author,
        "titel": titel,
        "jahr": jahr,
        "jahr008": jahr008
    }

In [ ]:
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

### Analyse

In [ ]:
# Häufigkeit der Jahre zählen
jahr_counts = df['jahr'].value_counts().head(15).reset_index()
jahr_counts.columns=['jahr', 'count']
jahr_counts

(Zum Bereinigen solcher Daten siehe: https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_Visualisierung_Tutorial.ipynb)

In [ ]:
# Jahreszahlen aus dem 008-Feld zählen: 
jahr008_counts = df['jahr008'].value_counts().head(15).reset_index()
jahr008_counts.columns=['jahr008', 'count']
jahr008_counts

In [ ]:
fig1 = px.bar(jahr_counts.sort_values('jahr'), x='jahr', y='count', height=550,
             title='Anzahl der Veröffentlichungen pro Jahr im Datenset',
             labels={'jahr': 'Jahr', 'count': 'Anzahl'})

# Diagramm anzeigen
fig1.show()

In [ ]:
fig2 = px.bar(jahr008_counts.sort_values('jahr008'), x='jahr008', y='count', height=550,
             title='Anzahl der Veröffentlichungen pro Jahr im Datenset',
             labels={'jahr008': 'Jahr', 'count': 'Anzahl'})

# Diagramm anzeigen
fig2.show()

Oder als Kreisdiagramm: 

In [ ]:
# Erstellen eines entsprechenden Kreisdiagramms: 
fig3 = px.pie(jahr008_counts, names='jahr008', values='count', height=550,
             title='Anzahl der Veröffentlichungen pro Jahr im Datenset als Tortendiagramm',
             labels={'jahr008': 'Jahr', 'count': 'Anzahl'})

# Ändern der Beschriftung auf den einzelnen Tortenstücken: 
#fig3.update_traces(
    #textinfo='label+percent')#,  # Anzeigen des Jahres sowie des Prozentsatzes
    #insidetextorientation='horizontal' ) # Texausrichtung der Tortenstücke

# Anzeigen des Diagramms:
fig3.show()

In [ ]:
fig3.write_html("fig3.html")